# DSA2040 END SEMESTER EXAM

## SECTION 1: DATA WAREHOUSING

In [12]:
# STEP 1: imports & constants
import os, random, sqlite3
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

CURRENT_DATE = datetime(2025, 8, 12) 
DB_PATH = "retail_dw.db"
DATA_CSV = "retail_synthetic.csv"
DDL_SQL_PATH = "warehouse.sql"
OLAP_SQL_PATH = "olap_queries.sql"
PLOT_PATH = "sales_by_country.png"


In [13]:
# STEP 2: star schema
DDL_SQL = """
DROP TABLE IF EXISTS SalesFact;
DROP TABLE IF EXISTS TimeDim;
DROP TABLE IF EXISTS ProductDim;
DROP TABLE IF EXISTS CustomerDim;

CREATE TABLE CustomerDim (
    CustomerID   INTEGER PRIMARY KEY,
    CustomerName TEXT,
    Country      TEXT
);

CREATE TABLE ProductDim (
    ProductID   INTEGER PRIMARY KEY,
    ProductName TEXT,
    Category    TEXT,
    UnitPrice   REAL
);

CREATE TABLE TimeDim (
    TimeID  INTEGER PRIMARY KEY,
    Date    TEXT,
    Month   INTEGER,
    Quarter INTEGER,
    Year    INTEGER
);

CREATE TABLE SalesFact (
    SalesID     INTEGER PRIMARY KEY AUTOINCREMENT,
    TimeID      INTEGER,
    CustomerID  INTEGER,
    ProductID   INTEGER,
    Quantity    INTEGER,
    SalesAmount REAL,
    FOREIGN KEY (TimeID)     REFERENCES TimeDim(TimeID),
    FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
    FOREIGN KEY (ProductID)  REFERENCES ProductDim(ProductID)
);
""".strip()

with open(DDL_SQL_PATH, "w", encoding="utf-8") as f:
    f.write(DDL_SQL + "\n")

print(f"Saved DDL to {DDL_SQL_PATH}")


Saved DDL to warehouse.sql


In [14]:
# STEP 3: generating synthetic data and saving as CSV
random.seed(42); np.random.seed(42)

products = [
    (1, "Smartphone", "Electronics", 499.0),
    (2, "Laptop", "Electronics", 999.0),
    (3, "Headphones", "Electronics", 59.0),
    (4, "T-Shirt", "Clothing", 19.0),
    (5, "Jeans", "Clothing", 39.0),
    (6, "Sneakers", "Clothing", 79.0),
    (7, "Blender", "Home", 45.0),
    (8, "Toaster", "Home", 35.0),
    (9, "Desk Lamp", "Home", 25.0),
    (10, "Novel Book", "Books", 12.0),
]
countries = ["United Kingdom","Germany","France","Netherlands","Norway","Spain","Italy","Ireland","Belgium","Portugal"]
customer_ids = list(range(10001, 10101)) 

def make_data(n_rows=1000):
    start = datetime(2023, 1, 1)
    end = CURRENT_DATE
    recs = []
    for _ in range(n_rows):
        inv = f"INV-{random.randint(100000,999999)}"
        product_id, name, category, unit_price = random.choice(products)
        qty = random.randint(1, 50)
        # adding a few bad rows to test cleaning
        if random.random() < 0.02: qty *= -1   # negative qty
        if random.random() < 0.02: unit_price = 0.0  # zero price

        inv_date = start + timedelta(days=random.randint(0, (end-start).days))
        cust = random.choice(customer_ids)
        country = random.choice(countries)
        stock = f"STK{product_id:04d}"
        recs.append([inv, stock, name, qty, inv_date, unit_price, cust, country, product_id, category])

    cols = ["InvoiceNo","StockCode","Description","Quantity","InvoiceDate","UnitPrice",
            "CustomerID","Country","ProductID","Category"]
    return pd.DataFrame(recs, columns=cols)

df = make_data(1000)
df.to_csv(DATA_CSV, index=False)
print(df.head(), "\n\nSaved CSV ->", DATA_CSV)


PermissionError: [Errno 13] Permission denied: 'retail_synthetic.csv'

In [ ]:
# STEP 4: cleaning & transforming
df = pd.read_csv(DATA_CSV, parse_dates=["InvoiceDate"])

# removing outliers
before = len(df)
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)].copy()
print("Dropped outliers:", before - len(df))

# Adding Total Sales Column
df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

# Printing last 12 months as of Aug 12, 2025
cutoff = CURRENT_DATE - timedelta(days=365)
df_last = df[df["InvoiceDate"] >= cutoff].copy()
print("Rows in last year:", len(df_last))
df_last.head()


Dropped outliers: 41
Rows in last year: 375


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductID,Category,TotalSales
2,INV-629903,STK0010,Novel Book,2,2024-12-19,12.0,10070,Italy,10,Books,24.0
3,INV-331148,STK0008,Toaster,38,2025-02-16,35.0,10021,Italy,8,Home,1330.0
5,INV-201414,STK0006,Sneakers,23,2025-01-17,79.0,10059,Belgium,6,Clothing,1817.0
6,INV-230889,STK0007,Blender,6,2024-09-25,45.0,10047,Portugal,7,Home,270.0
9,INV-802729,STK0005,Jeans,45,2024-09-15,39.0,10082,France,5,Clothing,1755.0


In [ ]:
# STEP 5: building dims and fact (in-memory)
# CustomerDim (simple name tag for demo)
cust_dim = (
    df_last[["CustomerID","Country"]]
    .drop_duplicates()
    .assign(CustomerName=lambda x: "Customer_" + x["CustomerID"].astype(str))
)[["CustomerID","CustomerName","Country"]]

prod_dim = (
    df_last[["ProductID","Description","Category","UnitPrice"]]
    .drop_duplicates()
    .rename(columns={"Description":"ProductName"})
    .sort_values("ProductID")
)

# TimeDim (unique dates)
time_dim = (
    df_last[["InvoiceDate"]].drop_duplicates().sort_values("InvoiceDate").reset_index(drop=True)
)
time_dim["TimeID"] = time_dim.index + 1
time_dim["Date"] = time_dim["InvoiceDate"].dt.date
time_dim["Month"] = time_dim["InvoiceDate"].dt.month
time_dim["Quarter"] = ((time_dim["InvoiceDate"].dt.month - 1)//3 + 1).astype(int)
time_dim["Year"] = time_dim["InvoiceDate"].dt.year

# SalesFact (map TimeID)
fact = df_last.merge(time_dim[["InvoiceDate","TimeID"]], on="InvoiceDate", how="left")
sales_fact = fact[["TimeID","CustomerID","ProductID","Quantity","TotalSales"]].copy()
sales_fact = sales_fact.rename(columns={"TotalSales":"SalesAmount"})

print("CustomerDim:", cust_dim.shape, "ProductDim:", prod_dim.shape, "TimeDim:", time_dim.shape, "SalesFact:", sales_fact.shape)


CustomerDim: (316, 3) ProductDim: (10, 4) TimeDim: (223, 6) SalesFact: (375, 5)


In [ ]:
# STEP 6: loading to SQLite
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

with sqlite3.connect(DB_PATH) as conn:
    cust_dim.to_sql("CustomerDim", conn, if_exists="replace", index=False)
    prod_dim.to_sql("ProductDim", conn, if_exists="replace", index=False)
    time_dim[["TimeID","Date","Month","Quarter","Year"]].to_sql("TimeDim", conn, if_exists="replace", index=False)
    sales_fact.to_sql("SalesFact", conn, if_exists="replace", index=False)

print("Loaded tables into", DB_PATH)


Loaded tables into retail_dw.db


In [ ]:
# STEP 7: OLAP queries
ROLLUP_SQL = """
SELECT cd.Country, td.Quarter, SUM(sf.SalesAmount) AS TotalSales
FROM SalesFact sf
JOIN CustomerDim cd ON sf.CustomerID = cd.CustomerID
JOIN TimeDim td ON sf.TimeID = td.TimeID
GROUP BY cd.Country, td.Quarter
ORDER BY cd.Country, td.Quarter;
""".strip()

DRILL_SQL = """
SELECT td.Year, td.Month, SUM(sf.SalesAmount) AS TotalSales
FROM SalesFact sf
JOIN CustomerDim cd ON sf.CustomerID = cd.CustomerID
JOIN TimeDim td ON sf.TimeID = td.TimeID
WHERE cd.Country = 'United Kingdom'
GROUP BY td.Year, td.Month
ORDER BY td.Year, td.Month;
""".strip()

SLICE_SQL = """
SELECT SUM(sf.SalesAmount) AS TotalElectronicsSales
FROM SalesFact sf
JOIN ProductDim pd ON sf.ProductID = pd.ProductID
WHERE pd.Category = 'Electronics';
""".strip()

with open(OLAP_SQL_PATH, "w", encoding="utf-8") as f:
    f.write(ROLLUP_SQL + "\n\n" + DRILL_SQL + "\n\n" + SLICE_SQL + "\n")

with sqlite3.connect(DB_PATH) as conn:
    df_roll = pd.read_sql_query(ROLLUP_SQL, conn)
    df_drill = pd.read_sql_query(DRILL_SQL, conn)
    df_slice = pd.read_sql_query(SLICE_SQL, conn)

print("ROLL-UP (head):\n", df_roll.head(), "\n")
print("DRILL-DOWN (UK head):\n", df_drill.head(), "\n")
print("SLICE (Electronics total):\n", df_slice, "\n")
print("Saved queries to", OLAP_SQL_PATH)


ROLL-UP (head):
    Country  Quarter  TotalSales
0  Belgium        1     86755.0
1  Belgium        2     89308.0
2  Belgium        3     45189.0
3  Belgium        4     72486.0
4   France        1    131520.0 

DRILL-DOWN (UK head):
    Year  Month  TotalSales
0  2024      8     40921.0
1  2024      9    133472.0
2  2024     10     26927.0
3  2024     11     28967.0
4  2024     12     58960.0 

SLICE (Electronics total):
    TotalElectronicsSales
0              1272759.0 

Saved queries to olap_queries.sql


In [ ]:
# STEP 8: bar chart of total sales by country (across all quarters)
agg = df_roll.groupby("Country", as_index=False)["TotalSales"].sum()

plt.figure(figsize=(9,5))
plt.bar(agg["Country"], agg["TotalSales"]) 
plt.xticks(rotation=30, ha="right")
plt.title("Total Sales by Country (Last 12 Months)")
plt.ylabel("Sales Amount")
plt.tight_layout()
plt.savefig(PLOT_PATH, dpi=200)
plt.close()

print( PLOT_PATH)


sales_by_country.png
